In [2]:
%pip install tensorflow
%pip install tensorflow.keras.models
%pip install tensorflow.keras.layers
%pip install scikeras

  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 3.8.0
    Uninstalling keras-3.8.0:
      Successfully uninstalled keras-3.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikeras 0.13.0 requires keras>=3.2.0, but you have keras 2.15.0 which is incompatible.
^C
object address  : 0x106b999c0
object refcount : 2
object type     : 0x100c7a7c8
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement tensorflow.keras.models (from versions: none)
ERROR: No matching distribution found for tensorflow.keras.models

[notice] A new release of pip is available: 24.1.1 -> 25.0.

In [ ]:
import matplotlib.pyplot as plt, numpy as np, pandas as pd, seaborn as sb

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.preprocessing import LabelEncoder, StandardScaler

# NN imports
from sklearn.model_selection import GridSearchCV, cross_val_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor

# ensemble 
from sklearn.ensemble import BaggingRegressor

In [ ]:
# import datasets and transformations (reusability purposes)
import sys
sys.path.append('../src')

from datasets import product_sales

In [ ]:
# # COULD BE USED FOR FEATURE ENGINEERING

# # Ensure 'OrderDate' and 'Ship_by_Date' are in datetime format
# merged_data['OrderDate'] = pd.to_datetime(merged_data['OrderDate'])
# merged_data['Ship_by_Date'] = pd.to_datetime(merged_data['Ship_by_Date'])

# # 'Lead_Time' in days
# merged_data['Lead_Time'] = (merged_data['Ship_by_Date'] - merged_data['OrderDate']).dt.days
# # missing values in 'Lead_Time'
# merged_data['Lead_Time'].fillna(merged_data['Lead_Time'].median(), inplace=True)

In [ ]:
from scipy import stats

# Define a function to compute z-scores within each product group
def compute_zscore(group, threshold=3):
    # Only compute z-score if there are at least 2 data points in the group
    if len(group) >= 2:
        group['z_score'] = np.abs(stats.zscore(group['OrderQuantity']))
    else:
        group['z_score'] = 0  # or np.nan if preferred
    return group

# Group by "ProductNumber" and compute z-scores for "OrderQuantity" within each group
df_grouped = product_sales.groupby('ProductNumber').apply(compute_zscore)

# Define your threshold for what constitutes an "unusually high" order
z_threshold = 3

# Filter for outliers (orders with a z_score greater than the threshold)
df_outliers = df_grouped[df_grouped['z_score'] > z_threshold]

# Display the results
print("Unusually high order quantities (per ProductNumber):")
print(df_outliers[['ProductNumber', 'OrderQuantity', 'z_score']])

print(f'Dropping: \n{df_outliers.count()}')
# Keep only rows where the z-score is within the acceptable range
df_cleaned = df_grouped[df_grouped['z_score'] <= z_threshold].drop(columns=['z_score'])

# Reset index for clarity
df_cleaned.reset_index(drop=True, inplace=True)

product_sales = df_cleaned


### Make Model

In [ ]:
from model_utils import * # evaluate_model, param grids and hyperparm tuning
from sklearn.impute import SimpleImputer

# define the features and target variable from 'product_sales'
X = product_sales[
    [
        'ProductNumber', 'order_month', 
        'prev_month_sales', 'prev_week_sales', 'prev_2_month_sales', 'prev_3_month_sales',
        'moving_avg_3m', 'moving_avg_6m', 'moving_avg_12m', 'moving_avg_18m',
        'var_3m', 'var_6m', 'var_12m', 'var_18m',
        'log_var_3m', 'log_var_6m', 'log_var_12m', 'log_var_18m',
        'yoy_growth', 'sales_2023', 'sales_2024'
    ]
]

y = product_sales['OrderQuantity']

# convert 'ProductNumber' to numeric values
label_encoder = LabelEncoder()
X['ProductNumber'] = label_encoder.fit_transform(X['ProductNumber'])

# handle NaNs by filling them with the mean
imputer = SimpleImputer(strategy="mean")
X_imputed = imputer.fit_transform(X)

# normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# train-test split
X_train_scaled, X_val_scaled, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# train the model with scaled features (Random Forest Regressor)
rf = RandomForestRegressor()
rf_params = find_best_hyperparameters(rf, param_grids(rf.__class__.__name__), X_train_scaled, y_train)

In [ ]:
rf = RandomForestRegressor(**rf_params)
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_val_scaled)

In [ ]:
evaluate_model(rf, X_scaled, y)
print('--------------------------------------------')

In [ ]:
# # train the model with scaled features (Decision Tree Regressor)
# dt = DecisionTreeRegressor()
# dt_params = find_best_hyperparameters(dt, param_grids(dt), X_train_scaled, y_train)

In [ ]:
# dt = DecisionTreeRegressor(**dt_params)
# dt.fit(X_train_scaled, y_train)
# y_pred_dt = dt.predict(X_val_scaled)

In [ ]:
# evaluate_model(dt, X_scaled, y)
# print('--------------------------------------------')

In [ ]:
# # train the model with scaled features (Linear Regression)
# lr = LinearRegression()
# lr_params = find_best_hyperparameters(lr, param_grids(lr), X_train_scaled, y_train)

In [ ]:
# lr = LinearRegression(**lr_params)
# lr.fit(X_train_scaled, y_train)
# y_pred_lr = lr.predict(X_val_scaled)

In [ ]:
# evaluate_model(lr, X_scaled, y)
# print('--------------------------------------------')

In [ ]:
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV

# Define the neural network model
def create_nn_model(input_shape):
    model = Sequential()
    model.add(Dense(64, input_dim=input_shape, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

# Wrap the Keras model with KerasRegressor
input_shape = X_train_scaled.shape[1]
nn_model = KerasRegressor(build_fn=create_nn_model, input_shape=input_shape, verbose=1)

# Perform RandomizedSearchCV instead of GridSearchCV
grid_search = RandomizedSearchCV(
    estimator=nn_model, 
    param_distributions=param_grids("KerasRegressor"),  # Ensure this returns the correct params
    n_iter=10,  # Limit iterations to avoid long runtimes
    cv=3,  # Reduce cross-validation folds for speed
    n_jobs=-1, 
    verbose=2
)

grid_search.fit(X_train_scaled, y_train)

# Get the best parameters
nn_params = grid_search.best_params_
print(f'Best Parameters: {nn_params}')

# Rebuild the model with best parameters
best_nn_model = create_nn_model(input_shape)
best_nn_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
best_nn_model.fit(X_train_scaled, y_train, epochs=nn_params['epochs'], batch_size=nn_params['batch_size'], validation_split=0.2, verbose=1)

# Predict using the best model
y_pred_nn = best_nn_model.predict(X_val_scaled)

# Evaluate the model
evaluate_model(best_nn_model, X_scaled, y)


Fitting 3 folds for each of 1 candidates, totalling 3 fits


/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequentia

Epoch 1/200
Epoch 1/200
Epoch 1/200
447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 668us/step - loss: 54960054272.0000 - mean_absolute_error: 107305.0859
Epoch 2/200
447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 672us/step - loss: 58125799424.0000 - mean_absolute_error: 107679.4531
Epoch 2/200
447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 676us/step - loss: 47843393536.0000 - mean_absolute_error: 104799.9375
Epoch 2/200
447/447 ━━━━━━━━━━━━━━━━━━━━ 0s 640us/step - loss: 53708136448.0000 - mean_absolute_error: 105981.3594
Epoch 3/200
447/447 ━━━━━━━━━━━━━━━━━━━━ 0s 636us/step - loss: 46536953856.0000 - mean_absolute_error: 103310.2969
Epoch 3/200
447/447 ━━━━━━━━━━━━━━━━━━━━ 0s 643us/step - loss: 56405835776.0000 - mean_absolute_error: 105914.1172
Epoch 3/200
447/447 ━━━━━━━━━━━━━━━━━━━━ 0s 638us/step - loss: 48884629504.0000 - mean_absolute_error: 100706.0781
Epoch 4/200
447/447 ━━━━━━━━━━━━━━━━━━━━ 0s 636us/step - loss: 41348210688.0000 - mean_absolute_error: 97159.2812
Epoch 4/200
447/447 ━━━━━━━━━━━━━━━━━━━━ 0s 640us/step - 

/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


671/671 ━━━━━━━━━━━━━━━━━━━━ 1s 601us/step - loss: 55010713600.0000 - mean_absolute_error: 105919.4922
Epoch 2/200
551/671 ━━━━━━━━━━━━━━━━━━━━ 0s 548us/step - loss: 51433431040.0000 - mean_absolute_error: 101801.0078

KeyboardInterrupt: 

In [ ]:
print('---------------------- TRAINING PERFORMANCE ----------------------')
evaluate_model(nn_model, X_train_scaled, y_train)
print('--------------------------------------------')

---------------------- TRAINING PERFORMANCE ----------------------
Evaluating model...
Model: KerasRegressor


/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


RuntimeError: Random ops require a seed to be set when determinism is enabled. Please set a seed before running the op, e.g. by calling tf.random.set_seed(1).

In [ ]:
print('---------------------- TEST PERFORMANCE ----------------------')
evaluate_model(nn_model, X_val_scaled, y_val)
print('--------------------------------------------')

In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 6))

# Plot the bar plot to compare the actual and predicted values
axes[0].bar(product_sales.loc[y_val.index, 'ProductNumber'], y_val, label='Actual', alpha=0.6)
axes[0].bar(product_sales.loc[y_val.index, 'ProductNumber'], y_pred_rf, label='Predicted', alpha=0.6)
axes[0].set_xlabel('Product Number')
axes[0].set_ylabel('Order Quantity')
axes[0].set_title('Actual vs Predicted Order Quantity')
axes[0].set_ylim(0, 2250000)
axes[0].legend()

# Plot the residual plot
residuals = y_val - y_pred_rf
axes[1].scatter(product_sales.loc[y_val.index, 'ProductNumber'], residuals, alpha=0.6)
axes[1].axhline(y=0, color='r', linestyle='--')
axes[1].set_xlabel('Product Number')
axes[1].set_ylabel('Residuals')
axes[1].set_title('Residual Plot')

# Show the plots
plt.tight_layout()
plt.show()

In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 6))

# Plot the bar plot to compare the actual and predicted values
axes[0].bar(product_sales.loc[y_val.index, 'ProductNumber'], y_val, label='Actual', alpha=0.6)
axes[0].bar(product_sales.loc[y_val.index, 'ProductNumber'], y_pred_nn, label='Predicted', alpha=0.6)
axes[0].set_xlabel('Product Number')
axes[0].set_ylabel('Order Quantity')
axes[0].set_title('Actual vs Predicted Order Quantity')
axes[0].set_ylim(0, 2250000)
axes[0].legend()

# Plot the residual plot
residuals = y_val - y_pred_nn
axes[1].scatter(product_sales.loc[y_val.index, 'ProductNumber'], residuals, alpha=0.6)
axes[1].axhline(y=0, color='r', linestyle='--')
axes[1].set_xlabel('Product Number')
axes[1].set_ylabel('Residuals')
axes[1].set_title('Residual Plot')

# Show the plots
plt.tight_layout()
plt.show()

### Ensemble Methods

In [ ]:
bagging_rf = BaggingRegressor(estimator=RandomForestRegressor(**rf_params), n_estimators=10, random_state=42)
bagging_rf.fit(X_train_scaled, y_train)
y_pred_bagging_rf = bagging_rf.predict(X_val_scaled)

evaluate_model(bagging_rf, X_scaled, y)
print('--------------------------------------------')

In [ ]:
# bagging_nn = BaggingRegressor(estimator=KerasRegressor(build_fn=create_nn_model, input_shape=input_shape, **nn_params), n_estimators=10, random_state=42)
# bagging_nn.fit(X_train_scaled, y_train)
# y_pred_bagging_nn = bagging_nn.predict(X_val_scaled)

# evaluate_model(bagging_nn, X_scaled, y)
# print('--------------------------------------------')

NN Bagging: 
Mean Absolute Error (MAE): 145788.9249
Mean Squared Error (MSE): 117073044061.8419
Root Mean Squared Error (RMSE): 342159.3840
R-squared (R²): 0.0013
--------------------------------------------

In [ ]:
from sklearn.ensemble import VotingRegressor

rf_nn_voting = VotingRegressor(estimators=[('rf', RandomForestRegressor(**rf_params)), ('nn', KerasRegressor(build_fn=create_nn_model, input_shape=input_shape, **nn_params))])
rf_nn_voting.fit(X_train_scaled, y_train)
y_pred_rf_nn_voting = rf_nn_voting.predict(X_val_scaled)

NameError: name 'RandomForestRegressor' is not defined

In [ ]:
print('---------------------- TRAINING PERFORMANCE ----------------------')
evaluate_model(rf_nn_voting, X_train_scaled, y_train)
print('--------------------------------------------')

---------------------- TRAINING PERFORMANCE ----------------------
Evaluating model...
Model: KerasRegressor
Epoch 1/200


/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


537/537 ━━━━━━━━━━━━━━━━━━━━ 1s 606us/step - loss: 56549109760.0000 - mean_absolute_error: 106572.3203
Epoch 2/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 575us/step - loss: 54503477248.0000 - mean_absolute_error: 104347.4453
Epoch 3/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 571us/step - loss: 46666813440.0000 - mean_absolute_error: 95601.1562
Epoch 4/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 571us/step - loss: 34381103104.0000 - mean_absolute_error: 81029.7891
Epoch 5/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 650us/step - loss: 23174387712.0000 - mean_absolute_error: 66673.6094
Epoch 6/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 583us/step - loss: 16803419136.0000 - mean_absolute_error: 57658.1055
Epoch 7/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 692us/step - loss: 14242857984.0000 - mean_absolute_error: 53682.1328
Epoch 8/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 583us/step - loss: 13184120832.0000 - mean_absolute_error: 51780.1445
Epoch 9/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 583us/step - loss: 12552768512.0000 - mean_absolute_er

/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


537/537 ━━━━━━━━━━━━━━━━━━━━ 1s 593us/step - loss: 52015882240.0000 - mean_absolute_error: 106131.7188
Epoch 2/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 572us/step - loss: 49556398080.0000 - mean_absolute_error: 103495.1016
Epoch 3/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 570us/step - loss: 40252215296.0000 - mean_absolute_error: 93104.6172
Epoch 4/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 570us/step - loss: 27280449536.0000 - mean_absolute_error: 77237.1875
Epoch 5/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 577us/step - loss: 17996025856.0000 - mean_absolute_error: 64135.5742
Epoch 6/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 577us/step - loss: 14206886912.0000 - mean_absolute_error: 57554.0352
Epoch 7/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 585us/step - loss: 12925565952.0000 - mean_absolute_error: 54506.6328
Epoch 8/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step - loss: 12300674048.0000 - mean_absolute_error: 52567.1680
Epoch 9/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - loss: 11843956736.0000 - mean_absolute_er

...

537/537 ━━━━━━━━━━━━━━━━━━━━ 1s 584us/step - loss: 56598376448.0000 - mean_absolute_error: 106734.1953
Epoch 2/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 570us/step - loss: 54633078784.0000 - mean_absolute_error: 104687.0234
Epoch 3/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 572us/step - loss: 47232143360.0000 - mean_absolute_error: 96732.1094
Epoch 4/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 566us/step - loss: 35431768064.0000 - mean_absolute_error: 83049.7812
Epoch 5/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step - loss: 24156526592.0000 - mean_absolute_error: 68719.7422
Epoch 6/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 572us/step - loss: 17271715840.0000 - mean_absolute_error: 58973.8711
Epoch 7/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 574us/step - loss: 14321679360.0000 - mean_absolute_error: 54292.1719
Epoch 8/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 573us/step - loss: 13130537984.0000 - mean_absolute_error: 52098.3438
Epoch 9/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 573us/step - loss: 12491070464.0000 - mean_absolute_er

/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


537/537 ━━━━━━━━━━━━━━━━━━━━ 1s 590us/step - loss: 50511671296.0000 - mean_absolute_error: 104650.2188
Epoch 2/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 697us/step - loss: 48519962624.0000 - mean_absolute_error: 102519.1797
Epoch 3/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 601us/step - loss: 40963784704.0000 - mean_absolute_error: 93859.3984
Epoch 4/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 597us/step - loss: 29467969536.0000 - mean_absolute_error: 79230.9688
Epoch 5/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 593us/step - loss: 19654146048.0000 - mean_absolute_error: 65027.1719
Epoch 6/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 596us/step - loss: 14789193728.0000 - mean_absolute_error: 56850.2383
Epoch 7/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 603us/step - loss: 13255980032.0000 - mean_absolute_error: 53354.1953
Epoch 8/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 598us/step - loss: 12683531264.0000 - mean_absolute_error: 51608.0898
Epoch 9/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 606us/step - loss: 12278816768.0000 - mean_absolute_er

/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


537/537 ━━━━━━━━━━━━━━━━━━━━ 1s 570us/step - loss: 46822723584.0000 - mean_absolute_error: 103437.1328
Epoch 2/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 553us/step - loss: 44740530176.0000 - mean_absolute_error: 101036.2422
Epoch 3/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 552us/step - loss: 37381574656.0000 - mean_absolute_error: 91851.7891
Epoch 4/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 554us/step - loss: 26625949696.0000 - mean_absolute_error: 76823.6250
Epoch 5/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 552us/step - loss: 17904375808.0000 - mean_absolute_error: 62837.8750
Epoch 6/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 554us/step - loss: 13846717440.0000 - mean_absolute_error: 54931.1797
Epoch 7/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 547us/step - loss: 12571671552.0000 - mean_absolute_error: 51711.3438
Epoch 8/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 555us/step - loss: 12074497024.0000 - mean_absolute_error: 50258.4570
Epoch 9/200
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 555us/step - loss: 11713511424.0000 - mean_absolute_er

In [ ]:
print('---------------------- TEST PERFORMANCE ----------------------')
evaluate_model(rf_nn_voting, X_val_scaled, y_val)
print('--------------------------------------------')

---------------------- TEST PERFORMANCE ----------------------
Evaluating model...
Model: KerasRegressor
Epoch 1/200


/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


135/135 ━━━━━━━━━━━━━━━━━━━━ 1s 675us/step - loss: 59631697920.0000 - mean_absolute_error: 109097.3750
Epoch 2/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 601us/step - loss: 59598868480.0000 - mean_absolute_error: 109060.0938
Epoch 3/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 613us/step - loss: 59420233728.0000 - mean_absolute_error: 108874.9219
Epoch 4/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 605us/step - loss: 58951393280.0000 - mean_absolute_error: 108398.1484
Epoch 5/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 609us/step - loss: 58072616960.0000 - mean_absolute_error: 107510.4297
Epoch 6/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 602us/step - loss: 56705122304.0000 - mean_absolute_error: 106130.1016
Epoch 7/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 641us/step - loss: 54808363008.0000 - mean_absolute_error: 104196.2891
Epoch 8/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 604us/step - loss: 52366233600.0000 - mean_absolute_error: 101668.0547
Epoch 9/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 601us/step - loss: 49376448512.0000 - mean_absol

/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


135/135 ━━━━━━━━━━━━━━━━━━━━ 1s 666us/step - loss: 63727677440.0000 - mean_absolute_error: 109504.7500
Epoch 2/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 595us/step - loss: 63691497472.0000 - mean_absolute_error: 109466.8984
Epoch 3/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 605us/step - loss: 63490830336.0000 - mean_absolute_error: 109277.4844
Epoch 4/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 608us/step - loss: 62954598400.0000 - mean_absolute_error: 108787.4062
Epoch 5/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 603us/step - loss: 61944909824.0000 - mean_absolute_error: 107876.9844
Epoch 6/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 606us/step - loss: 60347547648.0000 - mean_absolute_error: 106439.0078
Epoch 7/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 606us/step - loss: 58115833856.0000 - mean_absolute_error: 104413.0781
Epoch 8/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 599us/step - loss: 55269388288.0000 - mean_absolute_error: 101786.7891
Epoch 9/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 603us/step - loss: 51872067584.0000 - mean_absol

...

135/135 ━━━━━━━━━━━━━━━━━━━━ 1s 677us/step - loss: 60652453888.0000 - mean_absolute_error: 107775.3359 
Epoch 2/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 604us/step - loss: 60620431360.0000 - mean_absolute_error: 107739.8203
Epoch 3/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 601us/step - loss: 60443979776.0000 - mean_absolute_error: 107562.0312
Epoch 4/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 603us/step - loss: 59984998400.0000 - mean_absolute_error: 107109.9766
Epoch 5/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 608us/step - loss: 59133321216.0000 - mean_absolute_error: 106278.3281
Epoch 6/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 610us/step - loss: 57809948672.0000 - mean_absolute_error: 104988.1250
Epoch 7/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 602us/step - loss: 55969226752.0000 - mean_absolute_error: 103190.3359
Epoch 8/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 602us/step - loss: 53616664576.0000 - mean_absolute_error: 100865.1484
Epoch 9/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 608us/step - loss: 50792206336.0000 - mean_abso

/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


135/135 ━━━━━━━━━━━━━━━━━━━━ 1s 662us/step - loss: 53701234688.0000 - mean_absolute_error: 107443.9766
Epoch 2/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 606us/step - loss: 53671141376.0000 - mean_absolute_error: 107407.5703
Epoch 3/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 598us/step - loss: 53511434240.0000 - mean_absolute_error: 107228.0000
Epoch 4/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 612us/step - loss: 53111267328.0000 - mean_absolute_error: 106784.1641
Epoch 5/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 617us/step - loss: 52384940032.0000 - mean_absolute_error: 105986.3125
Epoch 6/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 612us/step - loss: 51265437696.0000 - mean_absolute_error: 104758.0938
Epoch 7/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 604us/step - loss: 49679392768.0000 - mean_absolute_error: 103002.2578
Epoch 8/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 605us/step - loss: 47637606400.0000 - mean_absolute_error: 100698.7500
Epoch 9/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 608us/step - loss: 45175021568.0000 - mean_absol

/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


135/135 ━━━━━━━━━━━━━━━━━━━━ 1s 668us/step - loss: 55438917632.0000 - mean_absolute_error: 108472.9922 
Epoch 2/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 603us/step - loss: 55404167168.0000 - mean_absolute_error: 108431.6641
Epoch 3/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 605us/step - loss: 55206907904.0000 - mean_absolute_error: 108225.7578
Epoch 4/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 607us/step - loss: 54691753984.0000 - mean_absolute_error: 107709.6641
Epoch 5/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 604us/step - loss: 53717749760.0000 - mean_absolute_error: 106747.4609
Epoch 6/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 603us/step - loss: 52178239488.0000 - mean_absolute_error: 105222.8906
Epoch 7/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 600us/step - loss: 50038652928.0000 - mean_absolute_error: 103074.4922
Epoch 8/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 605us/step - loss: 47305543680.0000 - mean_absolute_error: 100277.0156
Epoch 9/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 604us/step - loss: 44040605696.0000 - mean_abso

# Voting Ensemble

Mean Absolute Error (MAE): 134711.8210
Mean Squared Error (MSE): 116899234027.1837
Root Mean Squared Error (RMSE): 341905.2998
R-squared (R²): 0.0028
--------------------------------------------

In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 6))

# Plot the bar plot to compare the actual and predicted values
axes[0].bar(product_sales.loc[y_val.index, 'ProductNumber'], y_val, label='Actual', alpha=0.6)
axes[0].bar(product_sales.loc[y_val.index, 'ProductNumber'], y_pred_rf_nn_voting, label='Predicted', alpha=0.6)
axes[0].set_xlabel('Product Number')
axes[0].set_ylabel('Order Quantity')
axes[0].set_title('Actual vs Predicted Order Quantity')
axes[0].set_ylim(0, 2250000)
axes[0].legend()

# Plot the residual plot
residuals = y_val - y_pred_rf_nn_voting
axes[1].scatter(product_sales.loc[y_val.index, 'ProductNumber'], residuals, alpha=0.6)
axes[1].axhline(y=0, color='r', linestyle='--')
axes[1].set_xlabel('Product Number')
axes[1].set_ylabel('Residuals')
axes[1].set_title('Residual Plot')

# Show the plots
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

print(y_pred_rf_nn_voting.tolist())

plt.figure(figsize=(10, 6))

# scatter plot, actual vs predicted values
sns.scatterplot(x=y_val, y=y_pred_rf_nn_voting, alpha=0.4, color='blue', label='Predicted')  # Predicted values in blue

# actual values in red
sns.scatterplot(x=y_val, y=y_val, alpha=0.4, color='#f26d1d', label='Actual')

# reference line for perfect prediction
plt.plot([min(y_val), max(y_val)], [min(y_val), max(y_val)], '--', color='black', label="Perfect Prediction")

plt.xlabel("Actual Order Quantity")
plt.ylabel("Predicted Order Quantity")
plt.title(f"Overall RN + NN Voting - Predicted vs. Actual Order Quantity")
plt.legend()

plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 6))

product_sales['DetailedOrderDate'] = pd.to_datetime(
    product_sales['order_year'].astype(str) +
    product_sales['order_week'].astype(str).str.zfill(2) +
    product_sales['order_weekday'].astype(str),
    format='%Y%W%w'
)

# Bar plot, actual and predicted values over time
axes[0].bar(product_sales.loc[y_val.index, 'DetailedOrderDate'], y_val, label='Actual', alpha=0.6)
axes[0].bar(product_sales.loc[y_val.index, 'DetailedOrderDate'], y_pred_rf_nn_voting, label='Predicted', alpha=0.6)
axes[0].set_xlabel('Order Date')
axes[0].set_ylabel('Order Quantity')
axes[0].set_title(f'Overall RN + NN Voting - Actual vs Predicted Order Quantity')
axes[0].set_ylim(0, 3000000)
axes[0].legend()
axes[0].tick_params(axis='x', rotation=45)  # Rotate x-axis labels for readability

# Line plot, actual and predicted values over time
sorted_indices = product_sales.loc[y_val.index, 'DetailedOrderDate'].argsort()
sorted_dates = product_sales.loc[y_val.index, 'DetailedOrderDate'].iloc[sorted_indices]
sorted_actual = y_val.iloc[sorted_indices]
sorted_predicted = y_pred_rf_nn_voting[sorted_indices]

axes[1].plot(sorted_dates, sorted_actual, label='Actual', alpha=0.6, marker='o', linestyle='-')
axes[1].plot(sorted_dates, sorted_predicted, label='Predicted', alpha=0.6, marker='x', linestyle='--')
axes[1].set_xlabel('Order Date')
axes[1].set_ylabel('Order Quantity')
axes[1].set_title(f'Overall RN + NN Voting - Actual vs Predicted Order Quantity')
axes[1].set_ylim(0, 3000000)
axes[1].legend()
axes[1].tick_params(axis='x', rotation=45)

# Residual plot over time
residuals = y_val - y_pred_rf_nn_voting
axes[2].scatter(product_sales.loc[y_val.index, 'DetailedOrderDate'], residuals, alpha=0.6)
axes[2].axhline(y=0, color='r', linestyle='--')
axes[2].set_xlabel('Order Date')
axes[2].set_ylabel('Residuals')
axes[2].set_title(f'Overall RN + NN Voting - Residual Plot Over Time')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()